In [1]:
from pytorch_tabnet.pretraining import TabNetPretrainer
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np



d:\ads_prj_nckh\.conda\lib\site-packages\pytorch_tabnet\utils.py:5: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy


In [2]:
# !pip install torchinfo pytorch_tabnet pandas

In [3]:
df_train = pd.DataFrame(np.random.rand(800, 10), columns=[f"feature_{i}" for i in range(10)])
df_test = pd.DataFrame(np.random.rand(200, 10), columns=[f"feature_{i}" for i in range(10)])

# Và labels tương ứng
y_train = pd.Series(np.random.rand(800))
y_test = pd.Series(np.random.rand(200))

# Chuyển đổi DataFrame và Series sang mảng NumPy với kiểu dữ liệu float32
X_train = df_train.values.astype(np.float32)
X_test = df_test.values.astype(np.float32)
y_train = y_train.values.astype(np.float32)
y_test = y_test.values.astype(np.float32)


In [4]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Kiểm tra giá trị thiếu
print("X_train has missing values:", np.any(np.isnan(X_train)))
print("X_test has missing values:", np.any(np.isnan(X_test)))
print("y_train has missing values:", np.any(np.isnan(y_train)))
print("y_test has missing values:", np.any(np.isnan(y_test)))

X_train shape: (800, 10)
X_test shape: (200, 10)
y_train shape: (800,)
y_test shape: (200,)
X_train has missing values: False
X_test has missing values: False
y_train has missing values: False
y_test has missing values: False


In [5]:
tabnet_params = {
    "n_d": 16,
    "n_a": 16,
    "n_steps": 3,
    "n_shared": 2,
    "n_independent": 2,
    "gamma": 1.3,
    "epsilon": 1e-15,
    "momentum": 0.98,
    "mask_type": "sparsemax",
    "lambda_sparse": 1e-3
}


unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    **tabnet_params
)
 

# Huấn luyện mô hình tiền huấn luyện
unsupervised_model.fit(
    X_train,
    eval_set=[X_test],  # Đảm bảo eval_set là danh sách các tuple (X, y)
    pretraining_ratio=0.8,
    max_epochs=101,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

d:\ads_prj_nckh\.conda\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 50.45297| val_0_unsup_loss_numpy: 6.042570114135742|  0:00:00s
epoch 1  | loss: 29.11551| val_0_unsup_loss_numpy: 4.289559841156006|  0:00:00s
epoch 2  | loss: 19.78084| val_0_unsup_loss_numpy: 4.747350215911865|  0:00:00s
epoch 3  | loss: 14.06169| val_0_unsup_loss_numpy: 3.781209945678711|  0:00:00s
epoch 4  | loss: 10.04586| val_0_unsup_loss_numpy: 3.6191000938415527|  0:00:00s
epoch 5  | loss: 7.99131 | val_0_unsup_loss_numpy: 2.4802000522613525|  0:00:00s
epoch 6  | loss: 6.31865 | val_0_unsup_loss_numpy: 2.1048200130462646|  0:00:00s
epoch 7  | loss: 5.1697  | val_0_unsup_loss_numpy: 2.067650079727173|  0:00:00s
epoch 8  | loss: 3.96239 | val_0_unsup_loss_numpy: 5.050839900970459|  0:00:00s
epoch 9  | loss: 3.46388 | val_0_unsup_loss_numpy: 1.4152899980545044|  0:00:01s
epoch 10 | loss: 2.9027  | val_0_unsup_loss_numpy: 1.3602499961853027|  0:00:01s
epoch 11 | loss: 2.35654 | val_0_unsup_loss_numpy: 1.2529699802398682|  0:00:01s
epoch 12 | loss: 2.11235 | val_0_u

d:\ads_prj_nckh\.conda\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [6]:
# Truy cập vào mô hình TabNet bên trong
from torchinfo import summary

tabnet_model = unsupervised_model.network

# Sử dụng torchinfo để in ra kiến trúc mô hình
summary(tabnet_model, input_size=(1, 10))  # input_size dựa trên kích thước của dữ liệu

Layer (type:depth-idx)                                       Output Shape              Param #
TabNetPretraining                                            [1, 10]                   --
├─EmbeddingGenerator: 1-1                                    [1, 10]                   --
├─TabNetEncoder: 1-2                                         [1, 16]                   --
│    └─BatchNorm1d: 2-1                                      [1, 10]                   20
│    └─FeatTransformer: 2-2                                  [1, 32]                   4,352
│    │    └─GLU_Block: 3-1                                   [1, 32]                   2,944
│    └─ModuleList: 2-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-17                            --                        (recursive)
│    └─FeatTransformer: 2-6                                  --                        (recursive)
│    │    └─GLU_Block: 3-5                                   -

In [7]:
encoder = tabnet_model.encoder
decoder = tabnet_model.decoder

print("\nEncoder Summary:")
print(encoder)

print("\nDecoder Summary:")
print(decoder)


Encoder Summary:
TabNetEncoder(
  (initial_bn): BatchNorm1d(10, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  (initial_splitter): FeatTransformer(
    (shared): GLU_Block(
      (shared_layers): ModuleList(
        (0): Linear(in_features=10, out_features=64, bias=False)
        (1): Linear(in_features=32, out_features=64, bias=False)
      )
      (glu_layers): ModuleList(
        (0): GLU_Layer(
          (fc): Linear(in_features=10, out_features=64, bias=False)
          (bn): GBN(
            (bn): BatchNorm1d(64, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
          )
        )
        (1): GLU_Layer(
          (fc): Linear(in_features=32, out_features=64, bias=False)
          (bn): GBN(
            (bn): BatchNorm1d(64, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
          )
        )
      )
    )
    (specifics): GLU_Block(
      (glu_layers): ModuleList(
        (0-1): 2 x GLU_Layer(
          (fc): Linear(in_fea